In [ ]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType

import reverse_geocoder as rg



In [ ]:
df = spark.read.table('earthquake_event_silver')

In [ ]:
def get_country_code(lat,lon):
    coordinates = (float(lat),float(lon))
    return rg.search(coordinates)[0].get('cc')



In [ ]:
get_country_code_udf = udf(get_country_code, StringType())

In [ ]:
df_with_location = df.withColumn('country_code',get_country_code_udf(col('latitude'),col('longitude')))

In [ ]:
df_with_location_sig_class = df_with_location.withColumn('sig_class', when(col('sig') < 100,'Low').\
                                                        when((col('sig') > 100) & (col('sig') < 500), 'Moderate').\
                                                        otherwise("High"))
# df_with_location_sig_class.show()

In [ ]:
df_with_location_sig_class.write.mode('overwrite').saveAsTable('earthquake_events_gold')